In [1]:
import pandas as pd
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
import bokeh.transform as btr
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
from datetime import datetime 
import timeit

In [2]:
data = pd.read_csv("http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip", compression = "zip", encoding = "latin1")

In [3]:
pio.templates.default = "plotly_white"
pio.templates
print(np.__version__)

1.19.5


In [4]:
data 

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-18,1426fb,1,4,10,1,28,10,7,1,...,99,1,1,2,97,3,99,MÃ©xico,97,97
1,2021-01-18,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
2,2021-01-18,0d55c9,2,12,9,1,9,9,16,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2021-01-18,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
4,2021-01-18,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139016,2021-01-18,2856be,2,12,9,2,9,9,15,1,...,2,2,97,1,2,7,99,MÃ©xico,97,97
4139017,2021-01-18,215a65,2,12,9,1,9,9,9,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97
4139018,2021-01-18,32eab3,2,12,9,2,9,9,10,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97
4139019,2021-01-18,26313e,2,12,9,1,12,9,17,1,...,99,2,97,1,2,7,99,MÃ©xico,97,97


In [5]:
last_update = data["FECHA_ACTUALIZACION"].iloc[0]
date_last_update = datetime.strptime(last_update, '%Y-%m-%d')

if date_last_update.day < 10:
    day = "0" + str(date_last_update.day) 
else:
    day = str(date_last_update.day) 

if date_last_update.month < 10:
    month = "0" + str(date_last_update.month)
else:
    month = str(date_last_update.month)

str_last_update = day + "/" + month + "/" + str(date_last_update.year) 
str_last_update

'18/01/2021'

In [6]:
data[["FECHA_ACTUALIZACION", "FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF"]] #COLUMNA QUE ME INTERESA DataFrame de una columna -----> Serie

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF
0,2021-01-18,2020-03-29,2020-03-31,9999-99-99
1,2021-01-18,2020-03-30,2020-03-30,9999-99-99
2,2021-01-18,2020-03-24,2020-03-26,9999-99-99
3,2021-01-18,2020-03-27,2020-04-02,9999-99-99
4,2021-01-18,2020-03-26,2020-03-26,2020-03-30
...,...,...,...,...
4139016,2021-01-18,2021-01-04,2021-01-07,9999-99-99
4139017,2021-01-18,2021-01-04,2021-01-09,9999-99-99
4139018,2021-01-18,2021-01-05,2021-01-09,9999-99-99
4139019,2021-01-18,2021-01-07,2021-01-08,9999-99-99


In [7]:
data[['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN']]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN
0,2021-01-18,1426fb,1
1,2021-01-18,1c4583,2
2,2021-01-18,0d55c9,2
3,2021-01-18,1468a5,1
4,2021-01-18,043f64,2
...,...,...,...
4139016,2021-01-18,2856be,2
4139017,2021-01-18,215a65,2
4139018,2021-01-18,32eab3,2
4139019,2021-01-18,26313e,2


In [8]:
data.loc[10] #INFORMACION DE LA FILA 10

FECHA_ACTUALIZACION      2021-01-18
ID_REGISTRO                  092521
ORIGEN                            1
SECTOR                           12
ENTIDAD_UM                        9
SEXO                              1
ENTIDAD_NAC                       9
ENTIDAD_RES                       9
MUNICIPIO_RES                    17
TIPO_PACIENTE                     1
FECHA_INGRESO            2020-03-30
FECHA_SINTOMAS           2020-03-28
FECHA_DEF                9999-99-99
INTUBADO                         97
NEUMONIA                          2
EDAD                             49
NACIONALIDAD                      1
EMBARAZO                          2
HABLA_LENGUA_INDIG                2
INDIGENA                          2
DIABETES                          2
EPOC                              2
ASMA                              2
INMUSUPR                          2
HIPERTENSION                      2
OTRA_COM                          2
CARDIOVASCULAR                    2
OBESIDAD                    

In [9]:
data.loc[[1,3,6]]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1,2021-01-18,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2021-01-18,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
6,2021-01-18,13757c,1,12,15,1,15,15,106,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97


In [10]:
#Casos negativos, sospechosos y activos
data["CLASIFICACION_FINAL"].value_counts()

7    2075246
3    1574639
6     345140
1      72385
5      58932
4      10201
2       2478
Name: CLASIFICACION_FINAL, dtype: int64

In [11]:
#Casos negativos ---> 7
condicion_negativos = data["CLASIFICACION_FINAL"] == 7

In [12]:
condicion_negativos.value_counts()

True     2075246
False    2063775
Name: CLASIFICACION_FINAL, dtype: int64

In [13]:
 negativos = data[condicion_negativos] #Filtrando el data FRAME para casos negativos
 print("Casos Negativos =",len(negativos))

Casos Negativos = 2075246


In [14]:
# 1,2,3 CASOS CONFIRMADOS
confirmados = data[data["CLASIFICACION_FINAL"].isin([1,2,3])]
print("Casos confirmados =",len(confirmados))

Casos confirmados = 1649502


In [15]:
confirmados["CLASIFICACION_FINAL"].value_counts()

3    1574639
1      72385
2       2478
Name: CLASIFICACION_FINAL, dtype: int64

In [16]:
#CASOS SORPECHOSOS 6 --> sospechosos sin muestra", "sospechosos con posibilidad de resultado" y "sospechosos sin posibilidad de resultado".
sospechosos = data[data["CLASIFICACION_FINAL"] == 6]
print("Sospechosos Totales = ", len(sospechosos))

Sospechosos Totales =  345140


In [17]:
sospechosos_sin_muestra=data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==97)]
#sospechosos_sin_muestra
print("Sospechosos sin muestra =",len(sospechosos_sin_muestra))

Sospechosos sin muestra = 237614


In [18]:
sospechosos_con_posibilidad =data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==3)]
#sospechosos_sin_muestra
print("Sospechosos con posibilidad =",len(sospechosos_con_posibilidad))

Sospechosos con posibilidad = 24931


In [19]:
sospechosos_sin_posibilidad=data[(data["CLASIFICACION_FINAL"].isin([6,5,4]))&(data["RESULTADO_LAB"].isin([4]))]
print("Sospechosos sin posibilidad =", len(sospechosos_sin_posibilidad))

Sospechosos sin posibilidad = 151728


In [20]:
#Buscando defunciones 
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [21]:
#data["FECHA_DEF"].value_counts()

In [22]:
defunciones = confirmados[confirmados["FECHA_DEF"] != "9999-99-99"]
print("Defunciones =", len(defunciones))


Defunciones = 141248


In [23]:
grupos = confirmados.groupby("FECHA_INGRESO")  #agrupando por fecha
confirmados_por_fecha = grupos.size().to_frame("confirmados").reset_index()   #INICIALMENTE grupos.size() es una serie con la fecha como ind
confirmados_por_fecha

,FECHA_INGRESO,confirmados
0,2020-01-13,1
1,2020-01-28,1
2,2020-01-29,1
3,2020-02-10,1
4,2020-02-19,1
...,...,...
327,2021-01-14,13260
328,2021-01-15,10362
329,2021-01-16,4049
330,2021-01-17,1371


In [24]:
confirmados_por_fecha.dtypes

FECHA_INGRESO    object
confirmados       int64
dtype: object

In [25]:
confirmados_por_fecha["FECHA_INGRESO"][:3]

0    2020-01-13
1    2020-01-28
2    2020-01-29
Name: FECHA_INGRESO, dtype: object

In [26]:
confirmados_por_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")
confirmados_por_fecha

,FECHA_INGRESO,confirmados,fecha_ingreso
0,2020-01-13,1,2020-01-13
1,2020-01-28,1,2020-01-28
2,2020-01-29,1,2020-01-29
3,2020-02-10,1,2020-02-10
4,2020-02-19,1,2020-02-19
...,...,...,...
327,2021-01-14,13260,2021-01-14
328,2021-01-15,10362,2021-01-15
329,2021-01-16,4049,2021-01-16
330,2021-01-17,1371,2021-01-17


In [27]:
confirmados_por_fecha.dtypes

FECHA_INGRESO            object
confirmados               int64
fecha_ingreso    datetime64[ns]
dtype: object

In [28]:
defunciones_por_fecha = defunciones.groupby("FECHA_DEF").size().to_frame("defunciones").reset_index()
defunciones_por_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_fecha


,FECHA_DEF,defunciones,fecha_defuncion
0,2020-03-03,1,2020-03-03
1,2020-03-13,1,2020-03-13
2,2020-03-17,1,2020-03-17
3,2020-03-18,2,2020-03-18
4,2020-03-20,1,2020-03-20
...,...,...,...
303,2021-01-14,597,2021-01-14
304,2021-01-15,366,2021-01-15
305,2021-01-16,290,2021-01-16
306,2021-01-17,199,2021-01-17


In [29]:
sospechosos_por_fecha = sospechosos.groupby("FECHA_SINTOMAS").size().to_frame("sospechosos").reset_index()
sospechosos_por_fecha["fecha_sintomas"] = pd.to_datetime(sospechosos_por_fecha["FECHA_SINTOMAS"], format = "%Y-%m-%d")


In [30]:
bpl.output_notebook()

Loading BokehJS ...

In [31]:
# Ploting each line in different figures ...
def plotCases(abscisa, ordenada, x_title, y_title, g_title, paint ):
    fig_1 = px.line( x = abscisa, y = ordenada, title = g_title, width = 1500, height = 600) # labels parameter can be added with a dict

    fig_1.update_traces(marker = dict(color = paint), line = dict(color = paint, width = 2))
    fig_1.update_xaxes(showgrid = True, title = x_title)
    fig_1.update_yaxes(showgrid = True, title = y_title)
    fig_1.show()
    
plotCases(confirmados_por_fecha["fecha_ingreso"], confirmados_por_fecha["confirmados"], 
        "Fecha de registro", "Casos confirmados", "Casos de COVID-19 en México","#b4004e")
plotCases(defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones"], 
        "Fecha de defunción", "Defunciones", "Defunciones por COVID-19 en México" , "black")
# plotCases(sospechosos_por_fecha["fecha_sintomas"],sospechosos_por_fecha["sospechosos"], 
#         "Fecha del inicio de sintomas", "Casos sospechosos", "Casos sospechosos de COVID-19 en México", "#c67c00")


In [32]:
# Computing the comulative cases ...
def commulative_cases(data):
    data_vec = np.array(data)
    data_acumulados = np.cumsum(data_vec)    
    return pd.Series(data_acumulados)                           # Numpy array to pandas Serie

# Commulative confirmed cases ....
confirmados_por_fecha["casos_acumulados"] = commulative_cases(confirmados_por_fecha["confirmados"])
# Commulative Deaths  ....
defunciones_por_fecha["defunciones_acumuladas"] = commulative_cases(defunciones_por_fecha["defunciones"])

In [33]:
#Computing the average with the actual day plus the 6 previous days
def average_of_seven_days(serie , threshold = 25000):
    data_vect = np.array(serie)
    average = np.zeros(data_vect.size)
    counter = 0
    init = 0
    # ....
    index = np.nonzero(data_vect > threshold)
    data_vect[index] = 0
    #Computing the average
    for i in range(data_vect.size):
        if i >= 6:
            average[counter] = np.mean(data_vect[init:i+1])
            init += 1
        else:
            average[counter] = 1/7 * np.sum(data_vect[0:i+1])

        counter += 1

    return  pd.Series(np.round(average))   #Return a pandas Serie with rounded values

In [34]:
# confirmados_por_fecha["casos_promedio"] =  average_of_seven_days(confirmados_por_fecha["confirmados"])
# defunciones_por_fecha["defunciones_promedio"] = average_of_seven_days(defunciones_por_fecha["defunciones"])

In [35]:

# fig_bar1_conf = px.bar(x =confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
    
#                     labels= dict(x = "Fecha de registro", y = "Nuevos casos confirmados"), title="Nuevos casos de COVID-19 reportados diariamente", width = 900)
#fig_bar1_conf.update_traces( marker = dict(color = "#ff1744", opacity = 0.6))
# fig_bar2_def = px.bar(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"],
#                         labels = dict(x = "Fecha de defunción", y = "Defunciones"), title = "Nuevas defunciones por COVID-19 reportadas diariamente", width = 900)

# fig_bar2_def.update_traces(marker = dict(color = "black"))

# Bar plots for confirmed and death cases ...
def show_confirmed_cases(x_vect,y_vect, region, anomalia):

    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900

    fig_bar1 = go.Figure()
    fig_bar1.add_trace(go.Bar(x = x_vect, y = y_vect, 
                                marker = dict(color = "#ff1744", opacity = 0.6 ), name = "Nuevos casos"))
    #Average of seven days curve ...                             
    fig_bar1.add_trace(go.Scatter(x = x_vect, y =  average_of_seven_days(y_vect),
                                        mode = "lines", line = dict(color = "#b71c1c", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar1.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar1.update_yaxes(title = "Nuevos casos confirmados")
    fig_bar1.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevos casos de COVID-19 reportados diariamente en " + region)
    #Showing figure ... 
    fig_bar1.show()

def show_deaths(x_vect,y_vect,region, anomalia):
    
    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900
    # Death cases ...
    fig_bar2 = go.Figure()
    fig_bar2.add_trace(go.Bar(x = x_vect, y = y_vect,
                                marker = dict(color = "#546e7a", opacity = 0.75), name = "Nuevas defunciones"))
    #Average of seven days curve ...                         
    fig_bar2.add_trace(go.Scatter(x = x_vect, y =average_of_seven_days(y_vect, threshold = 2500),
                                        mode = "lines", line = dict(color = "black", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar2.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar2.update_yaxes(title = "Nuevas defunciones confirmadas")
    fig_bar2.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevas defunciones de COVID-19 reportadas diariamente en " + region)
    #Showing figure ...
    fig_bar2.show()

show_confirmed_cases(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["confirmados"], "México", False)
show_deaths(defunciones_por_fecha["fecha_defuncion"],defunciones_por_fecha["defunciones"],"México", False)

In [36]:
# Ploting the commulative confirmed and deaths cases ...

def acum_plot(x_conf_data, y_conf_data, x_def_data, y_def_data, region ):
    fig_acum = go.Figure()

    fig_acum.add_trace(go.Scatter(x = x_conf_data, y = y_conf_data,
                    mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

    fig_acum.add_trace(go.Scatter(x = x_def_data, y = y_def_data,
                    mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))

    fig_acum.update_xaxes(title = "Fecha de registro")
    fig_acum.update_yaxes(title = "Número de casos acumulados")
    fig_acum.update_layout(title = "Casos acumulados en " + region )
    fig_acum.show()


# fig_acum = go.Figure()

# fig_acum.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["casos_acumulados"],
#                     mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

# fig_acum.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones_acumuladas"],
#                     mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))



# fig_acum.update_xaxes(title = "Fecha de registro")
# fig_acum.update_yaxes(title = "Número de casos acumulados")
# fig_acum.update_layout(title = "Casos acumulados en México")
# fig_acum.show()

acum_plot(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["casos_acumulados"],
            defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones_acumuladas"], region = "México" )


In [37]:

# Ploting each line in the same figure ...

# fig_2 = px.line(confirmados_por_fecha, x = "fecha_sintomas", y = "confirmados", width = 1000, height= 300,
#                  labels= dict(fecha_sintomas = "Fecha", confirmados = "Número de personas", name = "Casos confirmados"))

fig_2 = go.Figure()

fig_2.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                mode = "lines",name = "Casos confirmados", line = dict(color = "#b4004e")) )
fig_2.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"], 
                mode = "lines",name = "Defunciones", line = dict(color = "black") ) )
fig_2.add_trace(go.Scatter(x = sospechosos_por_fecha["fecha_sintomas"], y = sospechosos_por_fecha["sospechosos"],
                mode = "lines", name = "Casos sospechosos", line = dict(color = "#c67c00")))

fig_2.update_xaxes(showgrid = True, title = "Fecha")
fig_2.update_yaxes(showgrid = True , title = "Número de personas")
fig_2.update_layout( width = 1200, height= 400 , title = "COVID-19 México")
fig_2.show()

In [38]:
# Distribution of cases per state ...................................
confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-18,1426fb,1,4,10,1,28,10,7,1,...,99,1,1,2,97,3,99,MÃ©xico,97,97
1,2021-01-18,1c4583,2,12,9,2,9,9,4,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
2,2021-01-18,0d55c9,2,12,9,1,9,9,16,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2021-01-18,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
4,2021-01-18,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139011,2021-01-18,347f37,1,4,9,1,20,15,33,2,...,2,1,3,1,1,3,99,MÃ©xico,97,2
4139012,2021-01-18,21f293,2,4,2,2,11,2,2,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
4139015,2021-01-18,2e8cb2,2,12,9,1,30,15,58,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
4139017,2021-01-18,215a65,2,12,9,1,9,9,9,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97


In [39]:
def time_usage(func):
    def wrapper(*args, **kwargs):
        beg_ts = time.time()
        retval = func(*args, **kwargs)
        end_ts = time.time()
        print("elapsed time: %f" % (end_ts - beg_ts))
        return retval
    return wrapper

In [40]:
#Grouping by state ENTIDAD_RES
confirmados_por_estado = confirmados.groupby("ENTIDAD_RES").size().to_frame("Casos confirmados").reset_index()
confirmados_por_estado

,ENTIDAD_RES,Casos confirmados
0,1,18519
1,2,38784
2,3,20907
3,4,7578
4,5,55256
5,6,8650
6,7,8786
7,8,38463
8,9,409551
9,10,27119


In [41]:
# Cargando el diccionario de estados
dics = pd.read_excel("https://github.com/Riloro/mexico_covid19_dashboard/blob/master/diccionarios/201128%20Catalogos_.xlsx?raw=true", sheet_name = None)
dics.keys()

dict_keys(['Catálogo ORIGEN', 'Catálogo SECTOR', 'Catálogo SEXO', 'Catálogo TIPO_PACIENTE', 'Catálogo SI_NO', 'Catálogo NACIONALIDAD', 'Catálogo RESULTADO_LAB', 'Catálogo RESULTADO_ANTIGENO', 'Catálogo CLASIFICACION_FINAL', 'Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

In [42]:
dic_estados = dics["Catálogo de ENTIDADES"]
dic_estados

,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,AS
1,2,BAJA CALIFORNIA,BC
2,3,BAJA CALIFORNIA SUR,BS
3,4,CAMPECHE,CC
4,5,COAHUILA DE ZARAGOZA,CL
5,6,COLIMA,CM
6,7,CHIAPAS,CS
7,8,CHIHUAHUA,CH
8,9,CIUDAD DE MÉXICO,DF
9,10,DURANGO,DG


In [43]:
confirmados_por_entidad = pd.merge(confirmados_por_estado, dic_estados, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD" ) # left outer join
confirmados_por_entidad

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,18519,1,AGUASCALIENTES,AS
1,2,38784,2,BAJA CALIFORNIA,BC
2,3,20907,3,BAJA CALIFORNIA SUR,BS
3,4,7578,4,CAMPECHE,CC
4,5,55256,5,COAHUILA DE ZARAGOZA,CL
5,6,8650,6,COLIMA,CM
6,7,8786,7,CHIAPAS,CS
7,8,38463,8,CHIHUAHUA,CH
8,9,409551,9,CIUDAD DE MÉXICO,DF
9,10,27119,10,DURANGO,DG


In [44]:
confirmados_por_entidad.dtypes

ENTIDAD_RES            int64
Casos confirmados      int64
CLAVE_ENTIDAD          int64
ENTIDAD_FEDERATIVA    object
ABREVIATURA           object
dtype: object

In [45]:
# Bar plot of the confirmed cases by state
confirmados_por_entidad.sort_values("Casos confirmados", ascending = False, inplace = True ) # Ordenando por número de casos de forma descendente 
fig_bar_edo = px.bar( x = confirmados_por_entidad["ENTIDAD_FEDERATIVA"], y = confirmados_por_entidad["Casos confirmados"],
                        labels = dict(x = "Entidad Federativa", y = "Número de casos confirmados"), title = "Número de casos confirmados por estado")


fig_bar_edo.update_xaxes(type='category', tickfont=dict(size = 10))
fig_bar_edo.update_traces(marker = dict(color = "#b71c1c"), opacity = 0.8)
fig_bar_edo.show()

In [46]:
stime = timeit.default_timer()
poblaciones = pd.read_csv("http://www.conapo.gob.mx/work/models/CONAPO/Datos_Abiertos/Proyecciones2018/pob_mit_proyecciones.csv", encoding="latin1")
e = timeit.default_timer() - stime
poblaciones
print(e)

195.7783666999967


In [47]:
#poblacion_2020 = poblaciones[poblaciones["AÑO"] == 2020]
stime = timeit.default_timer()
poblacion_2020 = pd.read_csv("https://raw.githubusercontent.com/Riloro/mexico_covid19_dashboard/master/diccionarios/poblacion2020_new.csv", encoding='latin1')
e = timeit.default_timer() - stime
print(e)

1.3414371000035317


In [48]:

start_time_new = timeit.default_timer()
pop2020 = pd.read_csv("https://raw.githubusercontent.com/Riloro/mexico_covid19_dashboard/master/diccionarios/poblacion2020_new.csv", encoding='latin1')
elapsed_new = timeit.default_timer() - start_time_new

print(elapsed_new)


1.5007227000023704


In [49]:
poblacion_2020

,RENGLON,AÑO,ENTIDAD,CVE_GEO,EDAD,SEXO,POBLACION
0,15401,2020,República Mexicana,0,0,Hombres,1087447
1,15402,2020,República Mexicana,0,0,Mujeres,1048275
2,15403,2020,República Mexicana,0,1,Hombres,1093379
3,15404,2020,República Mexicana,0,1,Mujeres,1054636
4,15405,2020,República Mexicana,0,2,Hombres,1100800
...,...,...,...,...,...,...,...
7255,585856,2020,Zacatecas,32,107,Mujeres,1
7256,585857,2020,Zacatecas,32,108,Hombres,0
7257,585858,2020,Zacatecas,32,108,Mujeres,0
7258,585859,2020,Zacatecas,32,109,Hombres,0


In [50]:
grupos_pob = poblacion_2020.groupby(["CVE_GEO","ENTIDAD"])
poblacion_tot_estado = grupos_pob["POBLACION"].sum().to_frame("POBLACION").reset_index()   # Sum operation in the POBLACION column of each group

In [51]:
datos_por_estado = pd.merge(confirmados_por_entidad, poblacion_tot_estado, how = "left", left_on = "CLAVE_ENTIDAD", right_on = "CVE_GEO" )
datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION
0,9,409551,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645
1,15,169105,15,MÉXICO,MC,15,México,17427790
2,11,95394,11,GUANAJUATO,GT,11,Guanajuato,6228175
3,19,94796,19,NUEVO LEÓN,NL,19,Nuevo León,5610153
4,14,60907,14,JALISCO,JC,14,Jalisco,8409693
5,26,57748,26,SONORA,SR,26,Sonora,3074745
6,5,55256,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720
7,21,54455,21,PUEBLA,PL,21,Puebla,6604451
8,27,49248,27,TABASCO,TC,27,Tabasco,2572287
9,30,47067,30,VERACRUZ DE IGNACIO DE LA LLAVE,VZ,30,Veracruz,8539862


In [52]:
poblacion_tot_estado

,CVE_GEO,ENTIDAD,POBLACION
0,0,República Mexicana,127792286
1,1,Aguascalientes,1434635
2,2,Baja California,3634868
3,3,Baja California Sur,804708
4,4,Campeche,1000617
5,5,Coahuila,3218720
6,6,Colima,785153
7,7,Chiapas,5730367
8,8,Chihuahua,3801487
9,9,Ciudad de México,9018645


In [53]:
lista_estados = list(datos_por_estado["ENTIDAD"])
print(lista_estados)

['Ciudad de México', 'México', 'Guanajuato', 'Nuevo León', 'Jalisco', 'Sonora', 'Coahuila', 'Puebla', 'Tabasco', 'Veracruz', 'San Luis Potosí', 'Tamaulipas', 'Baja California', 'Querétaro', 'Chihuahua', 'Michoacán', 'Oaxaca', 'Sinaloa', 'Hidalgo', 'Yucatán', 'Guerrero', 'Durango', 'Zacatecas', 'Baja California Sur', 'Aguascalientes', 'Quintana Roo', 'Morelos', 'Tlaxcala', 'Nayarit', 'Chiapas', 'Colima', 'Campeche']


In [54]:
#Datos por estado a CSV .....
datos_por_estado.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/poblaciones_casos_por_estado.csv",
                                            encoding="latin1", index=False)
# Rate of confirmed case by 100,000 people 
datos_por_estado["TASA DE CASOS POR 100K"] = 100000 * datos_por_estado["Casos confirmados"]/datos_por_estado["POBLACION"]
datos_por_estado.sort_values("TASA DE CASOS POR 100K", ascending = False, inplace = True)
datos_por_estado.reset_index(drop = True, inplace = True)

datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION,TASA DE CASOS POR 100K
0,9,409551,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645,4541.158899
1,3,20907,3,BAJA CALIFORNIA SUR,BS,3,Baja California Sur,804708,2598.085268
2,27,49248,27,TABASCO,TC,27,Tabasco,2572287,1914.560856
3,26,57748,26,SONORA,SR,26,Sonora,3074745,1878.139488
4,5,55256,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720,1716.707263
5,22,38615,22,QUERÉTARO,QT,22,Querétaro,2279637,1693.910039
6,19,94796,19,NUEVO LEÓN,NL,19,Nuevo León,5610153,1689.722188
7,24,45363,24,SAN LUIS POTOSÍ,SP,24,San Luis Potosí,2866142,1582.719907
8,11,95394,11,GUANAJUATO,GT,11,Guanajuato,6228175,1531.652531
9,10,27119,10,DURANGO,DG,10,Durango,1868996,1450.992940


In [55]:
# Bar plot of the rate of confirmed cases per 100k persons 
fig_bar_cases_rate = px.bar(x = datos_por_estado["ENTIDAD"], y = datos_por_estado["TASA DE CASOS POR 100K"],
                            labels = dict(y = "Casos confirmados / 100k personas" , x = "") ,title = "Tasa de casos confirmados por cada 100,000 personas")


fig_bar_cases_rate.update_traces(marker = dict(color = "#ff616f"))
#fig_bar_cases_rate.update_xaxes(tickfont = dict(size = 11))
fig_bar_cases_rate.show()

In [56]:
# Grouping by state ENTIDAD_RES and FECHA_INGRESO
confirmados_por_estado_fecha = confirmados.groupby(["ENTIDAD_RES","FECHA_INGRESO"]).size().to_frame("confirmados").reset_index()
#Confirmados por estado a CSV .....
confirmados_por_estado_fecha.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/confirmados_por_estado.csv", encoding="latin1",                                                   index=False)
confirmados_por_estado_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_estado_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")


confirmados_por_estado_fecha

,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso
0,1,2020-02-20,1,2020-02-20
1,1,2020-03-14,1,2020-03-14
2,1,2020-03-17,1,2020-03-17
3,1,2020-03-18,3,2020-03-18
4,1,2020-03-19,1,2020-03-19
...,...,...,...,...
9765,32,2021-01-13,145,2021-01-13
9766,32,2021-01-14,148,2021-01-14
9767,32,2021-01-15,104,2021-01-15
9768,32,2021-01-16,14,2021-01-16


In [57]:
defunciones_por_estado_fecha = defunciones.groupby(["ENTIDAD_RES", "FECHA_DEF"]).size().to_frame("defunciones").reset_index()
#Defunciones por estado a CSV ............................
defunciones_por_estado_fecha.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/defunciones_por_estado.csv",
                                            encoding="latin1", index=False)
defunciones_por_estado_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_estado_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_estado_fecha

,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion
0,1,2020-04-10,1,2020-04-10
1,1,2020-04-13,1,2020-04-13
2,1,2020-04-17,1,2020-04-17
3,1,2020-04-19,1,2020-04-19
4,1,2020-04-22,1,2020-04-22
...,...,...,...,...
8677,32,2021-01-13,10,2021-01-13
8678,32,2021-01-14,7,2021-01-14
8679,32,2021-01-15,5,2021-01-15
8680,32,2021-01-16,1,2021-01-16


In [58]:
entidad_clave = datos_por_estado["ENTIDAD"].to_frame()
entidad_clave["CLAVE_ENTIDAD"] = datos_por_estado["CLAVE_ENTIDAD"]
# Merge between entidad_cave and confirmados_por_estado_fecha
confirmados_por_estado_fecha_clave = pd.merge(confirmados_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")

In [59]:
# Merge betweem entidad_clave and defunciones_por_estado_fecha
defunciones_por_estado_fecha_clave = pd.merge(defunciones_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")

In [60]:
clave_estado = 32
confirmados_mich = confirmados_por_estado_fecha_clave[confirmados_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado]
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())
confirmados_mich.reset_index(drop = True, inplace = True)
confirmados_mich["confirmados_acumulados"] = commulative_cases(confirmados_mich["confirmados"]) # Accumulated confirmed cases in Mich.
confirmados_mich

Total de casos confirmados =  23311


,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso,ENTIDAD,CLAVE_ENTIDAD,confirmados_acumulados
0,32,2020-02-19,1,2020-02-19,Zacatecas,32,1
1,32,2020-03-03,1,2020-03-03,Zacatecas,32,2
2,32,2020-03-04,3,2020-03-04,Zacatecas,32,5
3,32,2020-03-11,1,2020-03-11,Zacatecas,32,6
4,32,2020-03-16,1,2020-03-16,Zacatecas,32,7
...,...,...,...,...,...,...,...
293,32,2021-01-13,145,2021-01-13,Zacatecas,32,23023
294,32,2021-01-14,148,2021-01-14,Zacatecas,32,23171
295,32,2021-01-15,104,2021-01-15,Zacatecas,32,23275
296,32,2021-01-16,14,2021-01-16,Zacatecas,32,23289


In [61]:
defunciones_mich = defunciones_por_estado_fecha_clave[defunciones_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado ]
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
defunciones_mich.reset_index(drop = True, inplace = True)     
defunciones_mich["defunciones_acumuladas"] = commulative_cases(defunciones_mich["defunciones"])
defunciones_mich

Total de defunciones acumuladas =  1947


,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion,ENTIDAD,CLAVE_ENTIDAD,defunciones_acumuladas
0,32,2020-03-31,1,2020-03-31,Zacatecas,32,1
1,32,2020-04-04,1,2020-04-04,Zacatecas,32,2
2,32,2020-04-19,1,2020-04-19,Zacatecas,32,3
3,32,2020-04-23,1,2020-04-23,Zacatecas,32,4
4,32,2020-04-26,1,2020-04-26,Zacatecas,32,5
...,...,...,...,...,...,...,...
255,32,2021-01-13,10,2021-01-13,Zacatecas,32,1932
256,32,2021-01-14,7,2021-01-14,Zacatecas,32,1939
257,32,2021-01-15,5,2021-01-15,Zacatecas,32,1944
258,32,2021-01-16,1,2021-01-16,Zacatecas,32,1945


In [62]:
#
acum_plot(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados_acumulados"],
            defunciones_mich["fecha_defuncion"], defunciones_mich["defunciones_acumuladas"], region = confirmados_mich["ENTIDAD"].iloc[0] )
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())

Total de defunciones acumuladas =  1947
Total de casos confirmados =  23311


In [63]:
show_confirmed_cases(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)
show_deaths(defunciones_mich["fecha_defuncion"],defunciones_mich["defunciones"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)

In [64]:
jhu_daily_reports = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-17-2021.csv", encoding="utf-8")
mexico_daily_report = jhu_daily_reports[jhu_daily_reports["Country_Region"] == "Mexico"]
mexico_daily_report

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
387,NaN,NaN,Aguascalientes,Mexico,2021-01-18 05:22:34,21.8853,-102.2916,18501,1498,0,17003.0,"Aguascalientes, Mexico",1289.596308,8.096860
388,NaN,NaN,Baja California,Mexico,2021-01-18 05:22:34,30.8406,-115.2838,38697,6289,0,32408.0,"Baja California, Mexico",1064.605372,16.251906
389,NaN,NaN,Baja California Sur,Mexico,2021-01-18 05:22:34,26.0444,-111.6661,20741,835,0,19906.0,"Baja California Sur, Mexico",2577.456668,4.025843
390,NaN,NaN,Campeche,Mexico,2021-01-18 05:22:34,19.8301,-90.5349,7552,989,0,6563.0,"Campeche, Mexico",754.734329,13.095869
391,NaN,NaN,Chiapas,Mexico,2021-01-18 05:22:34,16.7569,-93.1292,8766,1259,0,7507.0,"Chiapas, Mexico",152.974495,14.362309
392,NaN,NaN,Chihuahua,Mexico,2021-01-18 05:22:34,28.6330,-106.0691,38326,4653,0,33673.0,"Chihuahua, Mexico",1008.184429,12.140583
393,NaN,NaN,Ciudad de Mexico,Mexico,2021-01-18 05:22:34,19.4326,-99.1332,407254,19417,0,387837.0,"Ciudad de Mexico, Mexico",4515.689441,4.767786
394,NaN,NaN,Coahuila,Mexico,2021-01-18 05:22:34,27.0587,-101.7068,55081,4696,0,50385.0,"Coahuila, Mexico",1711.270319,8.525626
395,NaN,NaN,Colima,Mexico,2021-01-18 05:22:34,19.1223,-104.0072,8591,839,0,7752.0,"Colima, Mexico",1094.181644,9.766034
396,NaN,NaN,Durango,Mexico,2021-01-18 05:22:34,24.5593,-104.6588,27087,1724,0,25363.0,"Durango, Mexico",1449.280790,6.364677


In [65]:

start_time = timeit.default_timer()
#Casos confirmados reportados por JHU ....
jhu_confirmed_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", encoding = "utf-8")
jhu_confirmed_mexico = jhu_confirmed_time_serie[jhu_confirmed_time_serie["Country/Region"] == "Mexico"]
#Muertes reportadas por JHU ...
jhu_deaths_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",encoding="utf-8")
jhu_deaths_mexico = jhu_deaths_time_serie[jhu_deaths_time_serie["Country/Region"] == "Mexico"]
#Recuperados reportados por JHU ...
jhu_recovered_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv",encoding="utf-8")
jhu_recovered_mexico = jhu_recovered_time_serie[jhu_recovered_time_serie["Country/Region"] == "Mexico"]
elapsed = timeit.default_timer() - start_time
print(elapsed)

2.523255000000063


In [66]:
raw_confirmed_data = jhu_confirmed_mexico.reset_index(drop=True).transpose().reset_index()
raw_deats_data = jhu_deaths_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data = jhu_recovered_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data

,index,0
0,Province/State,NaN
1,Country/Region,Mexico
2,Lat,23.6345
3,Long,-102.5528
4,1/22/20,0
...,...,...
362,1/14/21,1185621
363,1/15/21,1199810
364,1/16/21,1212403
365,1/17/21,1223108


In [67]:
#Insert a character in the ith position
def insert_str(init_str, insert_str, i):
    return init_str[:i] + insert_str + init_str[i:]

In [68]:
def insert_commas(string):
    step = 3
    size = len(string)
    quotient = size//step
    mod = size % step
    
    if mod == 0:
        commas_number = quotient - 1
        init_index = step
    else:
        commas_number = quotient
        init_index = mod 

    for i in range(commas_number):
        string = insert_str(string, ",", init_index )
        init_index = init_index + step + 1
    return string

In [69]:
#Filtrando el numero de casos y fechas de cada dataFrame ...
jhu_conf_mex_df = raw_confirmed_data["index"].iloc[4:].to_frame("fecha_confirmados").reset_index(drop = True)
jhu_conf_mex_df["confirmados"] = raw_confirmed_data[0].iloc[4:].reset_index(drop = True)
jhu_conf_mex_df["fecha_confirmados_dateTime"] = pd.to_datetime(jhu_conf_mex_df["fecha_confirmados"], format = "%m/%d/%y")

jhu_deaths_mex_df = raw_deats_data["index"].iloc[4:].to_frame("fecha_muerte").reset_index(drop = True)
jhu_deaths_mex_df["muertes"] = raw_deats_data[0].iloc[4:].reset_index(drop = True)
jhu_deaths_mex_df["fecha_muerte_dateTime"] = pd.to_datetime(jhu_deaths_mex_df["fecha_muerte"], format = "%m/%d/%y")

jhu_recovered_mex_df = raw_recovered_data["index"].iloc[4:].to_frame("fecha_recuperado").reset_index(drop = True)
jhu_recovered_mex_df["recuperados"] = raw_recovered_data[0].iloc[4:].reset_index(drop = True)
jhu_recovered_mex_df["feha_recuperados_dateTime"] = pd.to_datetime(jhu_recovered_mex_df["fecha_recuperado"], format = "%m/%d/%y")

In [70]:
# Inserting commas ...
str_conf_mex = str(jhu_conf_mex_df["confirmados"].iloc[-1])
str_deaths_mex = str(jhu_deaths_mex_df["muertes"].iloc[-1])
str_recovered_mex = str(jhu_recovered_mex_df["recuperados"].iloc[-1])
#Calling inset_commas function ...
str_conf_mex_commas = insert_commas(str_conf_mex)
str_recovered_mex_commas = insert_commas(str_recovered_mex)
str_deaths_mex_commas = insert_commas(str_deaths_mex)
print(str_conf_mex_commas)
print(str_recovered_mex_commas)
print(str_deaths_mex_commas)

1,649,502
1,237,321
141,248


In [71]:
jhu_deaths_mex_df

,fecha_muerte,muertes,fecha_muerte_dateTime
0,1/22/20,0,2020-01-22
1,1/23/20,0,2020-01-23
2,1/24/20,0,2020-01-24
3,1/25/20,0,2020-01-25
4,1/26/20,0,2020-01-26
...,...,...,...
358,1/14/21,137916,2021-01-14
359,1/15/21,139022,2021-01-15
360,1/16/21,140241,2021-01-16
361,1/17/21,140704,2021-01-17


In [72]:
#Graficando los casos confirmados de muertes y confirmados ....
acum_plot(jhu_conf_mex_df["fecha_confirmados_dateTime"], jhu_conf_mex_df["confirmados"],
            jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes"], region="México")

In [73]:
# difference JHU data in order to get daily cases ....
def diff(data):
    v_1 = np.array(data)
    v_2 = np.diff(v_1)
    res = np.insert(v_2,0,v_1[0])

    return pd.Series(res)

In [74]:
#Getting daily cases ...
jhu_conf_daily = diff(jhu_conf_mex_df["confirmados"])   
jhu_conf_mex_df["casos diarios"] = jhu_conf_daily

#Getting daily deaths...
jhu_daily_deaths = diff(jhu_deaths_mex_df["muertes"]) 
jhu_deaths_mex_df["muertes diarias"] = jhu_daily_deaths

jhu_deaths_mex_df

,fecha_muerte,muertes,fecha_muerte_dateTime,muertes diarias
0,1/22/20,0,2020-01-22,0
1,1/23/20,0,2020-01-23,0
2,1/24/20,0,2020-01-24,0
3,1/25/20,0,2020-01-25,0
4,1/26/20,0,2020-01-26,0
...,...,...,...,...
358,1/14/21,137916,2021-01-14,999
359,1/15/21,139022,2021-01-15,1106
360,1/16/21,140241,2021-01-16,1219
361,1/17/21,140704,2021-01-17,463


In [75]:
jhu_conf_mex_df

,fecha_confirmados,confirmados,fecha_confirmados_dateTime,casos diarios
0,1/22/20,0,2020-01-22,0
1,1/23/20,0,2020-01-23,0
2,1/24/20,0,2020-01-24,0
3,1/25/20,0,2020-01-25,0
4,1/26/20,0,2020-01-26,0
...,...,...,...,...
358,1/14/21,1588369,2021-01-14,16468
359,1/15/21,1609735,2021-01-15,21366
360,1/16/21,1630258,2021-01-16,20523
361,1/17/21,1641428,2021-01-17,11170


In [76]:
 
 show_confirmed_cases(jhu_conf_mex_df["fecha_confirmados_dateTime"],jhu_conf_mex_df["casos diarios"], region = "México", anomalia = True)
 show_deaths(jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes diarias"],  region = "México", anomalia = True)